In [173]:
import torch
torch.__version__

'1.0.0.dev20181018'

In [174]:
bs = 8
nc = 4

In [175]:
y_pred = torch.randn(bs, nc)
y_pred = y_pred.sigmoid()
y_pred.shape

torch.Size([8, 4])

In [176]:
y_pred

tensor([[0.7382, 0.5250, 0.4268, 0.4756],
        [0.6568, 0.4793, 0.3691, 0.4151],
        [0.6520, 0.2794, 0.5058, 0.3773],
        [0.7320, 0.6620, 0.2627, 0.5787],
        [0.6108, 0.4781, 0.7272, 0.7101],
        [0.7471, 0.6282, 0.4449, 0.6647],
        [0.6908, 0.4687, 0.6190, 0.4050],
        [0.7219, 0.7927, 0.8400, 0.5160]])

In [177]:
y_true = torch.randint(low=0, high=2, size=(bs, nc))
y_true

tensor([[1, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [1, 1, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]])

In [178]:
y_rank = y_pred.argsort(descending=True)
y_rank

tensor([[0, 1, 3, 2],
        [0, 1, 3, 2],
        [0, 2, 3, 1],
        [0, 1, 3, 2],
        [2, 3, 0, 1],
        [0, 3, 1, 2],
        [0, 2, 1, 3],
        [2, 1, 0, 3]])

In [179]:
k = 10
nc = y_true.shape[1]
assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"

AssertionError: k (10) needs to be less than or equal to the number of classes (4)

In [188]:
k = 3
nc = y_true.shape[1]
assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"

In [181]:
y_true

tensor([[1, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [1, 1, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]])

In [182]:
y_true_totals = y_true.sum(dim=-1).float()
y_true_totals

tensor([2., 1., 2., 1., 1., 3., 1., 1.])

In [183]:
y_rank

tensor([[0, 1, 3, 2],
        [0, 1, 3, 2],
        [0, 2, 3, 1],
        [0, 1, 3, 2],
        [2, 3, 0, 1],
        [0, 3, 1, 2],
        [0, 2, 1, 3],
        [2, 1, 0, 3]])

In [184]:
y_reorder = torch.stack([y_true_i[y_rank[i]] for i,y_true_i in enumerate(y_true)])
y_reorder

tensor([[1, 0, 0, 1],
        [0, 0, 1, 0],
        [1, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 1, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 0]])

In [185]:
y_reorder_topk = y_reorder[:, :k]

In [186]:
recall_at_k = y_reorder_topk.sum(dim=-1).float() / y_true_totals
recall_at_k

tensor([0.5000, 1.0000, 1.0000, 1.0000, 1.0000, 0.6667, 1.0000, 1.0000])

In [202]:
recall_at_k.mean()

tensor(0.8958)

In [198]:
precision_at_k = y_reorder_topk.sum(dim=-1).float() / k
precision_at_k, precision_at_k.mean()

tensor([0.3333, 0.3333, 0.6667, 0.3333, 0.3333, 0.6667, 0.3333, 0.3333])

In [200]:
def precision_at_k(y_pred:Tensor, y_true:Tensor, k:int, sigmoid:bool=True):
    "Compute precision at `k` when `y_pred` and `y_true` are the same size."
    nc = y_true.shape[1]
    assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"    
    if sigmoid: y_pred = y_pred.sigmoid()
    y_rank = y_pred.argsort(descending=True)
    y_reorder_topk = torch.stack([y_true_i[y_rank[i]][:k] for i,y_true_i in enumerate(y_true)])
    return (y_reorder_topk.sum(dim=-1).float() / k).mean()

In [201]:
precision_at_k(y_pred, y_true, k)

tensor(0.4167)

In [205]:
def recall_at_k(y_pred:Tensor, y_true:Tensor, k:int, sigmoid:bool=True):
    "Compute recall at `k` when `y_pred` and `y_true` are the same size."
    nc = y_true.shape[1]
    assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"    
    if sigmoid: y_pred = y_pred.sigmoid()
    y_rank = y_pred.argsort(descending=True)
    y_true_totals = y_true.sum(dim=-1).float()
    y_reorder_topk = torch.stack([y_true_i[y_rank[i]][:k] for i,y_true_i in enumerate(y_true)])
    return (y_reorder_topk.sum(dim=-1).float() / y_true_totals).mean()

In [213]:
recall_at_k(y_pred, y_true, k)

tensor(0.8958)

In [208]:
from functools import partial

In [216]:
recall_at_3 = partial(recall_at_k, k=3)
recall_at_3(y_pred, y_true)

tensor(0.8958)

In [217]:
def recall_at_3(y_pred, y_true):
    return recall_at_k(y_pred, y_true, k=3)
recall_at_3(y_pred, y_true)

tensor(0.8958)